# Função responsável por gerar a chave através de uma password

A geração da chave é realizada com recurso a uma **KDF**, nomeadamente, a `PBKDF2HMAC`, que se encontra disponível no módulo **Cryptography**. Como argumento é passada a *password* introduzida pelo utilizador (em *bytes*) e um *salt* pseudoaleatório.

In [1]:
import os
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, hmac

def generate_key(password, salt=os.urandom(16)):
    backend = default_backend()
    
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=96,
        salt=salt,
        iterations=100000,
        backend=backend
    )

    key = kdf.derive(password)

    return key

# Função responsável por gerar o MAC de um criptograma

Para o efeito foi utilizado o `HMAC_SHA256` e a chave gerada para produzir um MAC que identifica unicamente o criptograma em questão. Por um lado, o emissor utiliza esta função para produzir o HMAC que é enviado juntamente com o criptograma. Por outro lado, o recetor utiliza-a com o objetivo de verificar o HMAC associado ao criptograma recebido e, consequentemente, verificar a integridade da mensagem.

In [2]:
def generate_mac(key, crypto):
    h = hmac.HMAC(key, hashes.SHA256(), backend = default_backend())
    h.update(crypto)
    return h.finalize()

# Função responsável por cifrar a mensagem a ser enviada

Para cifrar a mensagem a ser enviada é utilizado a cifra simétrica **AES** no modo **GCM**. Dado que este modo permite autenticar o conteúdo do argumento `associated_data`, este foi concretizado com o *salt* utilizado para gerar a chave usada. De salientar que a chave passada no argumento `key` tem um comprimento de 64 byte. Assim, os primeiros 32 byte são utilizados para cifrar o `plaintext` e os últimos 32 byte são usados como chave para gerar o HMAC do criptograma.

In [3]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from pickle import dumps

def encrypt(plaintext, key, associated_data):

    # Generate a random 96-bit IV.
    iv = os.urandom(12)
    
    # Construct an AES-GCM Cipher object with the 
    # given key and a randomly generated IV.
    encryptor = Cipher(algorithms.AES(key[:32]), 
                       modes.GCM(iv), 
                       backend=default_backend()).encryptor()
    
    # associated_data will be authenticated but not encrypted, 
    # it must also be passed in on decryption.
    encryptor.authenticate_additional_data(associated_data)

    # Encrypt the plaintext and get the associated ciphertext. 
    # GCM does not require padding.
    ciphertext = encryptor.update(plaintext.encode()) + encryptor.finalize()

    package = { 'iv': iv, 'tag': encryptor.tag, 'crypto': ciphertext }

    hmac = generate_mac(key[32:64], dumps(package))

    return {'mess' : package, 'tag' : hmac}

# Função responsável por decifrar a mensagem recebida

Para decifrar a mensagem recebido é realizado o processo inverso:
1. Retira-se o HMAC calculado no processo de cifragem.
2. Calcula-se o novo HMAC.
3. Os HMAC's são comparados.
4. Caso sejam iguais o processo continua. Caso contrário é devolvida uma mensagem de erro.
5. Decompõe-se o criptograma (iv + tag + crypto).
6. Verifica-se a autenticação da `associated_data`.
7. Decifra-se o criptograma.
8. Devolve-se o `plaintext`.

In [4]:
def decrypt(pkg, key, associated_data):
    crypto = pkg['mess']
    hmac = pkg['tag']
    
    macDest = generate_mac(key[32:64], dumps(crypto))
    if (hmac != macDest):
        return 'ERROR - MAC/Password is not equal'
    
    iv = crypto['iv']
    tag = crypto['tag']
    ciphertext = crypto['crypto']
    
    # Construct a Cipher object, with the key, iv, 
    # and additionally the GCM tag used for authenticating the message.
    decryptor = Cipher(algorithms.AES(key[:32]), 
                       modes.GCM(iv, tag), 
                       backend=default_backend()).decryptor()

    # We put associated_data back in or the tag will fail 
    # to verify when we finalize the decryptor.
    decryptor.authenticate_additional_data(associated_data)

    # Decryption gets us the authenticated plaintext. 
    # If the tag does not match an InvalidTag exception will be raised.
    plaintext = decryptor.update(ciphertext) + decryptor.finalize()

    return plaintext.decode()

# Processos que permitem a comunicação privada assíncrona entre um agente Emitter e um agente Receiver

1. O processo `Emitter`, para além de gerar a chave a partir da *password*, gera um HMAC para a chave gerada, cria uma mensagem, cifra essa mensagem com a chave e envia o criptograma (crypto + hmac_key + salt) pelo canal. 
2. O processo `Receiver` gera também a chave a partir de uma *password*, compara os HMAC's da chave e decifra a mensagem recebida pelo canal.
3. A classe `BiConn` tem como objetivo criar o `pipe` que vai ser usado pelos intervenientes para comunicação, e inicializar os processos que vão utilizar a via de diálogo criada.

In [5]:
from multiprocessing import Process, Pipe
from getpass import getpass
from base64 import b64encode, b64decode
import time
import lorem


def Emitter(conn):
    my_salt = os.urandom(16)
    passwd = getpass('Emmiter password: ').encode('utf-8') 
    # geração da chave a partir da password
    key = generate_key(passwd, my_salt) 
    hmac_key = generate_mac(key[64:], key)
    mess = lorem.sentence()
    print('Mensagem inicial: ' + mess)
    pkg = encrypt(mess, key, my_salt)
    pkg['hmac_key'] = hmac_key
    pkg['salt'] = my_salt
    conn.send(pkg) # envia uma mensagem pelo seu lado do Pipe
    conn.close()   # termina a ligação do seu lado do Pipe
    
def Receiver(conn):
    pkg = conn.recv()  # recebe a mensagem do seu lado do Pipe
    hmac_key = pkg['hmac_key']
    my_salt = pkg['salt']
    passwd = getpass('Emmiter password: ').encode('utf-8')
    # geração da chave a partir da password
    key = generate_key(passwd, my_salt) 
    if hmac_key == generate_mac(key[64:], key):
        mess = decrypt(pkg, key, my_salt)
        # faz qualquer coisa com a informação recebida
        print('Mensagem recebida: ' + mess) 
    else:
        print('ERROR - Different keys used.')
    conn.close()        # fecha a ligação do seu lado

class BiConn(object):
    def __init__(self, emitter, receiver, timeout=None):
        """
        emitter : a função que vai ligar ao lado esquerdo do Pipe
        receiver: a função que vai ligar ao outro lado
        timeout: (opcional) numero de segundos que aguarda 
                 pela terminação do processo
        """
        emitter_end, receiver_end = Pipe()
        self.timeout = timeout
        # os processos ligados ao Pipe
        self.eproc = Process(target=emitter, args=(emitter_end,))       
        self.rproc = Process(target=receiver, args=(receiver_end,))
        # as funções ligadas já ao Pipe
        self.emitter  = lambda : emitter(emitter_end)                       
        self.receiver = lambda : receiver(receiver_end)
    
    def auto(self, proc=None):
        if proc == None: # corre os dois processos independentes
            self.eproc.start()
            self.rproc.start()
            self.eproc.join(self.timeout)
            self.rproc.join(self.timeout)
        else:            # corre só o processo passado como parâmetro
            proc.start(); proc.join()
    
    #  corre as duas funções no contexto de um mesmo processo Python
    def manual(self):   
        self.emitter()
        self.receiver()
    
    
Conn = BiConn(Emitter, Receiver)
Conn.manual()

Emmiter password: ········
Mensagem inicial: Consectetur ipsum non dolorem.
Emmiter password: ········
Mensagem recebida: Consectetur ipsum non dolorem.
